<a href="https://colab.research.google.com/github/silvermind86/LLM_first_trial/blob/main/DesafioImers%C3%A3oAlura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**# Gerencia de endereços guiada por texto ou "voz" em um site de empréstimos**

Instalando o SDK do Gemini e iniciando funções e variáveis para uso futuro.

In [32]:
!pip install -q -U google-generativeai

import google.generativeai as genai
import numpy as np
import pandas as pd
import textwrap
import json
from google.colab import userdata
from IPython.display import display
from IPython.display import Markdown

safety_settings = {
    'HARASSMENT': 'BLOCK_LOW_AND_ABOVE',
    'HATE': 'BLOCK_MEDIUM_AND_ABOVE',
    'SEXUAL': 'BLOCK_LOW_AND_ABOVE',
    'DANGEROUS': 'BLOCK_MEDIUM_AND_ABOVE'
}
generation_config = {
    'candidate_count': 1,
    'temperature': 0.2
}
site_map = [
  {'titulo: ': 'Entrada', 'descrição': 'Entrada do site com boas vindas e links para as demais funções', 'url': 'http://banco.banco.banco/home'},
  {'titulo: ': 'Empréstimos para pessoa Física', 'descrição': 'Com base em seu CPF é possível realizar o empréstimo com base em seu salário e sua renda anual', 'url': 'http://banco.banco.banco/pessoa-fisica'},
  {'titulo: ': 'Empréstimos para pessoa Física - Transporte', 'descrição': 'Com base em seu salário atual podemos financiar a compra de automóveis', 'url': 'http://banco.banco.banco/pessoa-fisica/transporte'},
  {'titulo: ': 'Empréstimos para pessoa Física - Viagens', 'descrição': 'Com base em seu salário atual podemos adiantar a compra de passagens, estadia em hoteis e até aluguéis', 'url': 'http://banco.banco.banco/pessoa-fisica/viagem'},
  {'titulo: ': 'Empréstimos para pessoa Física - Morada', 'descrição': 'Com base em seu salário atual podemos finaciar casas, apartamentos e auxilar em obras e reformas', 'url': 'http://banco.banco.banco/pessoa-fisica/moradia'},
  {'titulo: ': 'Empréstimos para pessoa Física - Quitar dívidas', 'descrição': 'Com base em seu salário anual podemos renegociar suas dívidas com novos formatos de taxas e parcelamento', 'url': 'http://banco.banco.banco/pessoa-fisica/despesas'},
  {'titulo: ': 'Empréstimos para pessoa Jurídica', 'descrição': 'Com base em seu CNPJ é possível realizar o empréstimo no rendimento anual da sua empresa', 'url': 'http://banco.banco.banco/pessoa-juridica'},
  {'titulo: ': 'Empréstimos para pessoa Jurídica - Pequenos limites', 'descrição': 'Com base em seu rendimento anual podemos validar taxas menores para a empresa pagar variando as parcelas', 'url': 'http://banco.banco.banco/pessoa-juridica/pequenos-limites'},
  {'titulo: ': 'Empréstimos para pessoa Jurídica - Crédito vapor', 'descrição': 'Com base nos créditos a receber, podemos realizar os pagamentos do emprestimo como uma taxa em seus valores de recebiveis', 'url': 'http://banco.banco.banco/pessoa-juridica/vapor'},
  {'titulo: ': 'Empréstimos para pessoa Jurídica - Imóvel', 'descrição': 'Com base em seu CNPJ é possível financiar obras, reformas, contruções e compras de imóveis', 'url': 'http://banco.banco.banco/imovel'},
  {'titulo: ': 'Quem somos nós', 'descrição': 'Breve histórico da instituição e sua jornada da criação aos dias de hoje', 'url': 'http://banco.banco.banco/instituição'},
  {'titulo: ': 'Fale Conosco', 'descrição': 'Disponibiliza canais de comunicação como chats, e-mails e telefones de coloaborados que possam auxiliar em nossas matrizes e filiais', 'url': 'http://banco.banco.banco/contato'},
  {'titulo: ': 'Dúvidas mais frequentes', 'descrição': 'Um compilado de informações com maior recorrência solicitadas aos nossos colaboradores para facilitar atendimento de possíveis dúvidas.', 'url': 'http://banco.banco.banco/faq'},
]
prompt_header = 'Como se fosse um recepcionista de um banco, instrua o cliente com satisfação e alegria porém de forma bastante sucinta, respondendo a pergunta a seguir com as opções que o banco pode ajudar o cliente: '
sugestion_header = 'Como se fosse um atendente de um banco, reescreva de forma sucinta a frase: '

def to_markdown(text):
  text = text.replace('•', '  *')
  display(Markdown(textwrap.indent(text, '> ', predicate=lambda _: True)))


Iniciando as instâncias que serão utilizadas nas funções dos processos que serão criados.

In [35]:
genai.configure(api_key=userdata.get('api_key'))

df = pd.DataFrame(site_map)
df.columns = ['title', 'description', 'url']

def get_embed_value(title, description):
  return genai.embed_content(model='models/embedding-001',
                             content=description,
                             title=title,
                             task_type='RETRIEVAL_DOCUMENT')['embedding']

df['embeddings'] = df.apply(lambda row: get_embed_value(row['title'],row['description']), axis=1)

def query_prompt(prompt):
  query = genai.embed_content(model='models/embedding-001',
                              content=prompt,
                              task_type='RETRIEVAL_QUERY')['embedding']
  scalar = np.dot(np.stack(df['embeddings']), query)
  index = np.argmax(scalar)
  return df.iloc[index]

gemini = genai.GenerativeModel(model_name='gemini-1.0-pro',
                              generation_config=generation_config,
                              safety_settings=safety_settings)
chat = gemini.start_chat(history=[])


Criando o chat que retornará ao front-end o json que irá gerenciar a navegação da tela

In [ ]:
prompt = input('Olá, como podemos ajudar com seu empréstimo hoje?... ')

while prompt.lower() != 'sair':
  data = query_prompt(prompt)
  response = {
      'url': data['url'],
      'title': data['title'],
      'sugestion':chat.send_message(sugestion_header +  data['description']).text,
      'response': chat.send_message(prompt_header + prompt).text}
  print(json.dumps(response, ensure_ascii=False, indent=4))
  prompt = input('Existe algo a mais que eu possa lhe auxiliar quanto ao seu empréstimo?... ')